### Automated Concept Extraction

There are several steps to the automated concept extraction method that are outlined in their paper [here]().

Firstly, we need to create patches from images that represent the object we want to derive concepts for. This involves using skimage segmentation and extracting the patches and superpixels from this. These will be used to find visual features that can be used as concepts.

Once we have the patches that we need, we can make use of a clustering technique on the representations extracted from the bottleneck layers of our model after passing a patch through. This will allow us to find visually similar images that will hopefully group patches that represent the same visual concept.

These groups of patches can then be used to create a concept activation vector. This involves getting the activations of these ptaches relating to a concept and then random patches that as a group represent no descernable concept. A linear classifier is trained on these examples and the vector orthognal to the hyperplane that separates the concept examples from the random is taken to be the concept activation vector.

The influence of this concept can be determined by taking the partial derivative of the class logit you want to examine with respect to a bottleneck layer. Multiplying the CAV by this partial derivative will allow us to determine the impact this concept had on the prediction.

This concludes the rough overview of the method that will be employed. The aim is to create realistic, reasonable concepts from the mitotic figures without the need of manually gathering images of specific concepts.

### Importing libraries

In [1]:
import sys
import random
from pathlib import Path
import numpy as np
import sklearn.metrics as metrics
# from tcav import utils
import shutil
import torch
import torchvision.transforms as T
from PIL import Image

import Utils.ACE.ace_helpers as ace_helpers
from Utils.ACE.ace import ConceptDiscovery

### Testing with COCO

We will begin by taking some images from the COCO dataset, specifically those inclusing a tennis racket. We will use this example to test our method and ensure we are processing the images correctly and the results seem reasonable. This seems the best course of action as I have a better understanding of the visual features that concern a tennis racket and no formal understanding of the visual features of a mitotic figure.

In [2]:
# Create an output directory for our data
output = Path.cwd() / "ACE_COCO_output/"

# Create the relevant sub-directories.
discovered_concepts_dir = output / 'concepts/'
results_dir = output / 'results/'
cavs_dir = output / 'cavs/'
activations_dir = output / 'acts/'
results_summaries_dir = output / 'results_summaries/'

# # If the directory exists we delete it to generate new output.
# if output.exists():
#     shutil.rmtree(output)

# # Make all of the directories
# output.mkdir()
# discovered_concepts_dir.mkdir()
# results_dir.mkdir()
# cavs_dir.mkdir()
# activations_dir.mkdir()
# results_summaries_dir.mkdir()

In [3]:
# Specify the target class and the source directory.
target_class = "tennis racket"
source_dir = "D:\DS\DS4\Project\COCO"

In [4]:
%%capture
# Random concept for statistical testing.
random_concept = 'random_discovery'

# Define the bottleneck layers.

# Create the model variable and set it to evaluate.
mymodel = ace_helpers.MyModel("tmp", ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1'])
mymodel.model.eval()

In [5]:
mymodel.model.model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

### Selecting the bottleneck layers

In order to extract the activations and gradients from a layer, we need to determine which layer(s) are bottleneck layers. A bottleneck layer typically reduces the number of channels in the data between the input and output while keeping the size of the image equal by using a kernel of (1,1) and a stride of (1,1). This means that the model compresses the representation of the input in this layer and keeps the most important features for performing the task. This makes it the ideal layer for using the activations from to cluster the patches for ACE and to train the linear classifier for TCAV.

Looking at the model structure from above we can see that there are several such layers in the backbone of our model. It may be worth just taking a selection of these. I have decided to take the bottleneck from the last bottleneck unit in each layer. This means I will be using the following 4 layers.

```
bottleneck_layers = ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1']
```

These will be the layers I extract both the activations from and the gradients when looking at the influence of each concept.

In [6]:
# Creating the ConceptDiscovery class instance.
cd = ConceptDiscovery(
    mymodel,
    target_class,
    random_concept,
    ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1'],
    source_dir,
    activations_dir,
    cavs_dir,
    num_random_exp=2,
    channel_mean=True,
    max_imgs=100,
    min_imgs=50,
    num_discovery_imgs=100,
    num_workers=0)

We have initialized the ConceptDiscovery class, which contains the methods for creating superpixels, clustering to find concepts, creating concept activation vectors and testing these. We will make use of this class for most of the notebook.

In [ ]:
# Creating the dataset of image patches.
cd.create_patches(discovered_concepts_dir, param_dict={'n_segments': [15]})

# Saving the concept discovery target class images.
image_dir = discovered_concepts_dir / 'images'
image_dir.mkdir()
ace_helpers.save_images(image_dir.absolute(),
                        (cd.discovery_images * 256).astype(np.uint8))

We can now check the output directory to find the raw discovery images, the superpixels and the patches. These can now be used to find potential visual concepts.

In [22]:
# Discovering Concepts
cd.discover_concepts(discovered_concepts_dir, method='KM', param_dicts={'n_clusters': 25})

Calculating activations for superpixels: 100%|███████████████████████████████████████| 130/130 [03:37<00:00,  1.67s/it]


After running the above code we have collected the activations from passing the superpixels through our model. After optionally getting the average across filters to reduce size and then flattening, we are ready to cluster. This is also carried out above, leaving us with a folder of images for each of these potential concepts. These can be seen in our output folder.

In [23]:
# Save discovered concept images (resized and original sized)
ace_helpers.save_concepts(cd, discovered_concepts_dir)

In [24]:
# TODO add to helper function generate_random
superpixels = discovered_concepts_dir / "superpixels"
list_of_files = list(superpixels.iterdir())

# Random selection of the the superpixels for random concept?
random.seed(42)
cd.random_imgs = np.array(random.sample(list_of_files, 50))

In [25]:
# Save the random imgs for review save_random
for img in cd.random_imgs:
    destination = img.parent.parent / "Random"
    destination.mkdir(exist_ok=True)
    
    shutil.copy(img, destination / img.name)

In [ ]:
cav_accuracies = cd.cavs()

Calculating activations for superpixels:  38%|████████████████▏                         | 5/13 [00:08<00:13,  1.64s/it]

In [ ]:
cav_accuracies

In [7]:
files = Path("D:/DS/DS4/Project/2023-ca4021-tegarta2/ACE_COCO_output/concepts/superpixels") #/ target_class
files_for_tcav = np.array(list(files.iterdir())[:5])

In [8]:
files_for_tcav[1:5]

array([WindowsPath('D:/DS/DS4/Project/2023-ca4021-tegarta2/ACE_COCO_output/concepts/superpixels/000_001.png'),
       WindowsPath('D:/DS/DS4/Project/2023-ca4021-tegarta2/ACE_COCO_output/concepts/superpixels/000_002.png'),
       WindowsPath('D:/DS/DS4/Project/2023-ca4021-tegarta2/ACE_COCO_output/concepts/superpixels/000_003.png'),
       WindowsPath('D:/DS/DS4/Project/2023-ca4021-tegarta2/ACE_COCO_output/concepts/superpixels/000_004.png')],
      dtype=object)

In [9]:
grads = cd._return_gradients(files_for_tcav)

Calculating gradients: 100%|█████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.08it/s]


In [20]:
grads["000_000.png"]["backbone.body.layer1.2.conv1"][0].shape

(40000,)

In [12]:
activations = cd._get_activations(files_for_tcav)

Calculating activations for superpixels: 100%|███████████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]


In [15]:
activations["backbone.body.layer1.2.conv1"].shape

(5, 40000)

In [21]:
vector = cd.load_cav_direction("tennis racket_concept1", "Random", "body.layer1.2.conv1")

AttributeError: 'NoneType' object has no attribute 'get_cav'

In [ ]:
vector.shape

In [ ]:
# Calculating CAVs and TCAV scores
cav_accuracies = cd.cavs(min_acc=0.0)
scores = cd.tcavs(test=False)
ace_helpers.save_ace_report(cd, cav_accuracies, scores,
                            results_summaries_dir + 'ace_results.txt')

In [ ]:
# Plot examples of discovered concepts
for bn in cd.bottlenecks:
    ace_helpers.plot_concepts(cd, bn, 10, address=results_dir)
# Delete concepts that don't pass statistical testing
cd.test_and_remove_concepts(scores)